In [1]:
import pandas as pd

import numpy as np

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC


from utils import remove_top_quantile


import pickle

In [2]:
# load data
df = pd.read_csv('../data/GiveMeSomeCredit-training.csv')
df = df.drop(["Unnamed: 0"], axis=1)

# impute missing values
imputer = SimpleImputer(strategy='median')

columns = df.columns
index = df.index    
df = pd.DataFrame(imputer.fit_transform(df))
df.columns = columns
df.index = index

# outlier removal
df = remove_top_quantile(df, "RevolvingUtilizationOfUnsecuredLines", 0.99)
df = remove_top_quantile(df, "DebtRatio", 0.99)
df = remove_top_quantile(df, "MonthlyIncome", 0.99)

# feature engineering
def f(a):
    # never late
    if a["NumberOfTime30-59DaysPastDueNotWorse"] == 0 and \
        a["NumberOfTime60-89DaysPastDueNotWorse"] == 0 and \
        a["NumberOfTimes90DaysLate"] == 0:
            return 0
    # 30-59 late
    if a["NumberOfTime30-59DaysPastDueNotWorse"] != 0 and \
        a["NumberOfTime60-89DaysPastDueNotWorse"] == 0 and \
        a["NumberOfTimes90DaysLate"] == 0:
            return 1
    # 60-89 late
    if a["NumberOfTime30-59DaysPastDueNotWorse"] != 0 and \
        a["NumberOfTime60-89DaysPastDueNotWorse"] != 0 and \
        a["NumberOfTimes90DaysLate"] == 0:
            return 2
    # 90+ late
    return 3


df["PastDueSevereness"] = df.apply(f, axis=1)

# drop
df = df.drop(["NumberOfTime30-59DaysPastDueNotWorse", "NumberOfTime30-59DaysPastDueNotWorse", "NumberOfTime30-59DaysPastDueNotWorse"], axis=1)

# scaling
scaler = StandardScaler()
df[['age',
    'NumberOfDependents',
    'MonthlyIncome',
    'DebtRatio', 
    'RevolvingUtilizationOfUnsecuredLines', 
    'NumberOfOpenCreditLinesAndLoans', 
    'NumberRealEstateLoansOrLines'
]] = scaler.fit_transform(
    df[['age',
        'NumberOfDependents',
        'MonthlyIncome',
        'DebtRatio', 
        'RevolvingUtilizationOfUnsecuredLines', 
        'NumberOfOpenCreditLinesAndLoans', 
        'NumberRealEstateLoansOrLines'
    ]])


In [4]:
# load data
df_test = pd.read_csv('../data/GiveMeSomeCredit-testing.csv')
df_test = df_test.drop(["Unnamed: 0"], axis=1)

# impute
columns = df_test.columns
index = df_test.index    
df_test = pd.DataFrame(imputer.transform(df_test))
df_test.columns = columns
df_test.index = index

# apply new feature
df_test["PastDueSevereness"] = df_test.apply(f, axis=1)

# drop
df_test = df_test.drop(["NumberOfTime30-59DaysPastDueNotWorse", "NumberOfTime30-59DaysPastDueNotWorse", "NumberOfTime30-59DaysPastDueNotWorse"], axis=1)

# scale
df_test[['age',
    'NumberOfDependents',
    'MonthlyIncome',
    'DebtRatio', 
    'RevolvingUtilizationOfUnsecuredLines', 
    'NumberOfOpenCreditLinesAndLoans', 
    'NumberRealEstateLoansOrLines'
]] = scaler.transform(
    df_test[['age',
        'NumberOfDependents',
        'MonthlyIncome',
        'DebtRatio', 
        'RevolvingUtilizationOfUnsecuredLines', 
        'NumberOfOpenCreditLinesAndLoans', 
        'NumberRealEstateLoansOrLines'
    ]])

In [5]:
X_train = df[[
    "RevolvingUtilizationOfUnsecuredLines",
    "age",	
    "DebtRatio",	
    "MonthlyIncome",	
    "NumberOfOpenCreditLinesAndLoans",	
    "NumberRealEstateLoansOrLines", 
    "NumberOfDependents", 	
    "PastDueSevereness"
]]

y_train = df[["SeriousDlqin2yrs"]]

X_test = df_test[[
    "RevolvingUtilizationOfUnsecuredLines",
    "age",	
    "DebtRatio",	
    "MonthlyIncome",	
    "NumberOfOpenCreditLinesAndLoans",	
    "NumberRealEstateLoansOrLines", 
    "NumberOfDependents", 	
    "PastDueSevereness"
]]

y_test = df_test[["SeriousDlqin2yrs"]]

### Baseline logistic regression model

In [6]:
logreg = LogisticRegression()

logreg.fit(X_train, y=y_train["SeriousDlqin2yrs"])

y_pred = logreg.predict(X_train)

accuracy = accuracy_score(y_train, y_pred)
precision = precision_score(y_train, y_pred)
recall = recall_score(y_train, y_pred)
f1 = f1_score(y_train, y_pred)

print(f"{accuracy=}, {precision=}, {recall=}, {f1=}")

accuracy=0.9354417594800162, precision=0.4725519287833828, recall=0.13775951557093424, f1=0.21332886805090423


We can see very low recall rate, it stems from class imbalance. Lets try to make a model with "balanced" class_weight parameter in an attempt to tackle class imbalance.

In [7]:
logreg = LogisticRegression(class_weight="balanced")

logreg.fit(X_train, y=y_train["SeriousDlqin2yrs"])

y_pred = logreg.predict(X_train)

accuracy = accuracy_score(y_train, y_pred)
precision = precision_score(y_train, y_pred)
recall = recall_score(y_train, y_pred)
f1 = f1_score(y_train, y_pred)

print(f"{accuracy=}, {precision=}, {recall=}, {f1=}")

accuracy=0.7979360051668579, precision=0.20331400653383172, recall=0.7469723183391004, f1=0.31962984384037013


With balanced class weights we have much more restrictive model. Recall is significantly higher while precision is halved. F1 score still remains quite low.